In [1]:
using Turing
using LimberJack
using CSV
using NPZ
using Plots
using FITSIO
using Plots
using LinearAlgebra
using Distributed
using StatsPlots
using BenchmarkTools

In [2]:
Threads.nthreads()

4

In [3]:
files = npzread("../data/DESY1_cls/Cls_meta.npz")
Cls_meta = cls_meta(files);

In [4]:
cov_tot = files["cov"]
inv_cov_tot = files["inv_cov"]
data_vector = files["cls"];

In [5]:
nuisances = Dict("b0" => 1.41,
                 "b1" => 1.62,
                 "b2" => 1.60,
                 "b3" => 1.92,
                 "b4" => 2.00)

cosmology = LimberJack.Cosmology(0.25, 0.05, 0.67, 0.96, 0.81,
                                 tk_mode="EisHu",
                                 Pk_mode="Halofit");

In [6]:
@benchmark Theory(cosmology, nuisances, Cls_meta, files)

BenchmarkTools.Trial: 
  memory estimate:  706.21 MiB
  allocs estimate:  43274841
  --------------
  minimum time:     1.355 s (5.99% GC)
  median time:      1.388 s (5.88% GC)
  mean time:        1.390 s (6.02% GC)
  maximum time:     1.427 s (5.64% GC)
  --------------
  samples:          4
  evals/sample:     1

In [1]:
@time Theory(cosmology, nuisances, Cls_meta, files)

LoadError: UndefVarError: Theory not defined

In [7]:
@benchmark Theory_parallel(cosmology, nuisances, Cls_meta, files)

BenchmarkTools.Trial: 
  memory estimate:  706.21 MiB
  allocs estimate:  43274883
  --------------
  minimum time:     1.218 s (12.52% GC)
  median time:      1.228 s (13.82% GC)
  mean time:        1.231 s (13.86% GC)
  maximum time:     1.245 s (13.63% GC)
  --------------
  samples:          5
  evals/sample:     1

In [8]:
@model function model_parallel(data_vector; cov_tot=cov_tot)
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 ~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)
    
    b0 = 1.69 #1.41 #~ Uniform(1.0, 3.0)
    b1 = 2.05 #1.62 #~ Uniform(1.0, 3.0)
    b2 = 2.01 #1.60 #~ Uniform(1.0, 3.0)
    b3 = 2.46 #1.92 #~ Uniform(1.0, 3.0)
    b4 = 2.54 #2.00 #~ Uniform(1.0, 3.0)
    
    #dz_g0 ~ Normal(0.008, 0.007)
    #dz_g1 ~ Normal(-0.005, 0.007)
    #dz_g2 ~ Normal(0.006, 0.006)
    #dz_g3 ~ Normal(0.0, 0.010)
    #dz_g4 ~ Normal(0.0, 0.010)
    
    #dz_k0 ~ Normal(-0.001, 0.016)
    #dz_k1 ~ Normal(-0.019, 0.013)
    #dz_k2 ~ Normal(-0.009, 0.011)
    #dz_k3 ~ Normal(-0.018, 0.022)
    
    #m0 ~ Normal(0.0, 0.035)
    #m1 ~ Normal(0.0, 0.035)
    #m2 ~ Normal(0.0, 0.035)
    #m3 ~ Normal(0.0, 0.035)
    
    #A_IA ~ Uniform(-5, 5) 
    #alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("b0" => b0,
                     "b1" => b1,
                     "b2" => b2,
                     "b3" => b3,
                     "b4" => b4) #,
                     #"dz_g0" => dz_g0,
                     #"dz_g1" => dz_g1,
                     #"dz_g2" => dz_g2,
                     #"dz_g3" => dz_g3,
                     #"dz_g4" => dz_g4,
                     #"dz_k0" => dz_k0,
                     #"dz_k1" => dz_k1,
                     #"dz_k2" => dz_k2,
                     #"dz_k3" => dz_k3,
                     #"m0" => m0,
                     #"m1" => m1,
                     #"m2" => m2,
                     #"m3" => m3,
                     #"A_IA" => A_IA,
                     #"alpha_IA" => alpha_IA)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = vcat(Theory_parallel(cosmology, nuisances, Cls_meta, files)...)
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [9]:
@model function model_linear(data_vector; cov_tot=cov_tot)
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 ~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)
    
    b0 = 1.69 #1.41 #~ Uniform(1.0, 3.0)
    b1 = 2.05 #1.62 #~ Uniform(1.0, 3.0)
    b2 = 2.01 #1.60 #~ Uniform(1.0, 3.0)
    b3 = 2.46 #1.92 #~ Uniform(1.0, 3.0)
    b4 = 2.54 #2.00 #~ Uniform(1.0, 3.0)
    
    #dz_g0 ~ Normal(0.008, 0.007)
    #dz_g1 ~ Normal(-0.005, 0.007)
    #dz_g2 ~ Normal(0.006, 0.006)
    #dz_g3 ~ Normal(0.0, 0.010)
    #dz_g4 ~ Normal(0.0, 0.010)
    
    #dz_k0 ~ Normal(-0.001, 0.016)
    #dz_k1 ~ Normal(-0.019, 0.013)
    #dz_k2 ~ Normal(-0.009, 0.011)
    #dz_k3 ~ Normal(-0.018, 0.022)
    
    #m0 ~ Normal(0.0, 0.035)
    #m1 ~ Normal(0.0, 0.035)
    #m2 ~ Normal(0.0, 0.035)
    #m3 ~ Normal(0.0, 0.035)
    
    #A_IA ~ Uniform(-5, 5) 
    #alpha_IA ~ Uniform(-5, 5)

    nuisances = Dict("b0" => b0,
                     "b1" => b1,
                     "b2" => b2,
                     "b3" => b3,
                     "b4" => b4) #,
                     #"dz_g0" => dz_g0,
                     #"dz_g1" => dz_g1,
                     #"dz_g2" => dz_g2,
                     #"dz_g3" => dz_g3,
                     #"dz_g4" => dz_g4,
                     #"dz_k0" => dz_k0,
                     #"dz_k1" => dz_k1,
                     #"dz_k2" => dz_k2,
                     #"dz_k3" => dz_k3,
                     #"m0" => m0,
                     #"m1" => m1,
                     #"m2" => m2,
                     #"m3" => m3,
                     #"A_IA" => A_IA,
                     #"alpha_IA" => alpha_IA)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = vcat(Theory_parallel(cosmology, nuisances, Cls_meta, files)...)
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [10]:
iterations = 100
nadapts = 20
TAP = 0.60
nchains = 4

4

In [11]:
chains_linear = sample(model_linear(data_vector), NUTS(TAP, nadapts),
                       iterations, progress=true, save_state=true)

LoadError: BoundsError: attempt to access Float64 at index [2]

In [12]:
chains_parallel = sample(model_parallel(data_vector), NUTS(TAP, nadapts),
                       iterations, progress=true, save_state=true)

LoadError: BoundsError: attempt to access Float64 at index [2]